In [25]:
from datetime import datetime

now = datetime.now()
print (now)

2021-01-07 11:34:58.739034


In [26]:
# 03 정보 추출하기

now = datetime.now()
print (now.year)
print (now.month)
print (now.day)
print(now.hour)
print(now.minute)
print(now.second)

2021
1
7
11
34
58


In [27]:
# 04 날짜 정보 다듬기

now = datetime.now()
print (str(now.month) + "/" + str(now.day) + "/" + str(now.year))

1/7/2021


In [28]:
# 05 시간 정보 다듬기

now = datetime.now()
print (str(now.hour) + ":" + str(now.minute) + ":" + str(now.second))
#print (str(now.month) + "/" + str(now.day) + "/" + str(now.year))

11:34:58


# 영업일 캘린더 (KRX)

* 한국거래소 홈페이지에서 휴장일을 xls로 받을 수 있다
* 휴장일 데이터로 부터 개장일(market days)를 생성
* 해외 시장(CME, EUREX)등은  pandas-market-calendars를 활용

<img width="320" src="http://i.imgur.com/ClIQff8.jpg" >

#### 2017 http://fb.com/financedata , http://financedata.github.io
<!-- TEASER_END -->

# 한국거래소 홈페이지 휴장일 조회
한국거래소 홈페이지 http://marketdata.krx.co.kr 에서 아래 메뉴에서 휴장일을 조회할 수 있다

#### 시장정보 > 시장동향 > 공시 > 증시일정 > 휴장일

링크는 아래와 같다

http://marketdata.krx.co.kr/contents/MKD/01/0110/01100305/MKD01100305.jsp

<img width="80%" src="http://i.imgur.com/xUGXh8f.png" >

한국거래소, CME, EUREX 캘린더를 년도별로 제공하고 있으며, '다운로드'를 누르면 엑셀(data.xls)파일로 다운로드 할 수 있다.

다운로드한 파일(data.xls)을 아래와 같이 공유파일 링크로 만들었다.

- 데이터 https://goo.gl/JGse1n

In [1]:
import pandas as pd

df_hdays = pd.read_excel('https://goo.gl/JGse1n')
df_hdays

,일자 및 요일,요일구분,비고
0,2019-01-01,화요일,신정
1,2019-02-04,월요일,설날
2,2019-02-05,화요일,설날
3,2019-02-06,수요일,설날
4,2019-03-01,금요일,삼일절
5,2019-05-01,수요일,근로자의날
6,2019-05-06,월요일,어린이날(대체휴일)
7,2019-06-06,목요일,현충일
8,2019-08-15,목요일,광복절
9,2019-09-12,목요일,추석


날짜 데이터에 요일이 함께 들어있어 가공이 필요하다.


In [30]:
# hdays = df_hdays['일자 및 요일'].str.extract('(\d{4}-\d{2}-\d{2})', expand=False)
hdays = df_hdays['일자 및 요일']
hdays

0     2019-01-01
1     2019-02-04
2     2019-02-05
3     2019-02-06
4     2019-03-01
5     2019-05-01
6     2019-05-06
7     2019-06-06
8     2019-08-15
9     2019-09-12
10    2019-09-13
11    2019-10-03
12    2019-10-09
13    2019-12-25
14    2019-12-31
Name: 일자 및 요일, dtype: object

In [31]:
hdays = pd.to_datetime(hdays)
hdays.name = '날짜'
print(type(hdays))
hdays

<class 'pandas.core.series.Series'>


0    2019-01-01
1    2019-02-04
2    2019-02-05
3    2019-02-06
4    2019-03-01
5    2019-05-01
6    2019-05-06
7    2019-06-06
8    2019-08-15
9    2019-09-12
10   2019-09-13
11   2019-10-03
12   2019-10-09
13   2019-12-25
14   2019-12-31
Name: 날짜, dtype: datetime64[ns]

# 개장일(market day)

2019년 중에 개장일(market day)목록을 만들어 보자. 우선 pandas.date_range() 함수를 사용하여 1월 1일~ 12월 31까지 날짜를 생성한다. (365일)

In [32]:
mdays = pd.date_range('2019-01-01', '2019-12-31')
mdays

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08',
               '2019-01-09', '2019-01-10',
               ...
               '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
               '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=365, freq='D')

이때 freq 를 'B'로 지정(business day)하면, 주말(토,일)은 제외된다. (260일)

날짜 생성에 대해서는 다음 URL을 참고 하자. 
https://pandas.pydata.org/pandas-docs/stable/timeseries.html

특히, 간격 별칭(Offset Aliases)에 대해서는 아래 표에 정리되어 있다.

https://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases

In [33]:
mdays = pd.date_range('2019-01-01', '2019-12-31', freq='B')
mdays

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10',
               '2019-01-11', '2019-01-14',
               ...
               '2019-12-18', '2019-12-19', '2019-12-20', '2019-12-23',
               '2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=261, freq='B')

여기서 휴일을 제외(df.drop)하면, 모든 영업일(market day)가 만들어진다. (244일)

In [34]:
mdays = mdays.drop(hdays)
mdays

DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-14', '2019-01-15',
               ...
               '2019-12-16', '2019-12-17', '2019-12-18', '2019-12-19',
               '2019-12-20', '2019-12-23', '2019-12-24', '2019-12-26',
               '2019-12-27', '2019-12-30'],
              dtype='datetime64[ns]', length=246, freq=None)

# DataFrame에 휴장일 적용하기

DataFrame에서 개장일(market day)만을 추출해해야 하는 경우를 살펴보자.

다음과 같은 데이터를 가정해 보자. (5월 1일~5월 30일, 1~30까지 values로 할당)

In [3]:
data = {'values': range(1,32)}
data

{'values': range(1, 32)}

In [36]:
df_sample = pd.DataFrame(data, index=pd.date_range('2019-05-01', '2019-05-31'))
df_sample.head(10)

,values
2019-05-01,1
2019-05-02,2
2019-05-03,3
2019-05-04,4
2019-05-05,5
2019-05-06,6
2019-05-07,7
2019-05-08,8
2019-05-09,9
2019-05-10,10


위 데이터 중에서 개장일만 추출하고자 한다. DateTimeIndex를 가진 두 개의 DataFrame의 교집합을 구하는 간단한 방법으로 pd.merge()을 쓸 수 있다.

In [37]:
# 개장일을 index로 갖는 DataFrame
df_mdays = pd.DataFrame(index=mdays)
df_mdays.head()

""
2019-01-02
2019-01-03
2019-01-04
2019-01-07
2019-01-08


In [38]:
# 두 DataFrame (df_sample, df_mdays)의 인덱스를 기준으로 합친다(merge)
df = pd.merge(df_sample, df_mdays, right_index=True, left_index=True)
df.head(20)

,values
2019-05-02,2
2019-05-03,3
2019-05-07,7
2019-05-08,8
2019-05-09,9
2019-05-10,10
2019-05-13,13
2019-05-14,14
2019-05-15,15
2019-05-16,16


이 경우, 두 인덱스의 공동부분(intersection)만 남으므로 개장일만 남게 된다.

# CME, EUREX 휴장일

영업일 캘린더 메뉴에서 한국거래소(KRX)외에 CME, EUREX와 캘린더도 제공하며, 엑셀로 다운로드 한 2017년 데이터는 다음과 같다.

* hdays-2017-CME.xls ( https://goo.gl/cvnhy2 ) 
* hdays-2017-EUREX.xls ( https://goo.gl/s2PgBx ) 

In [39]:
pd.read_excel('https://goo.gl/cvnhy2')

,구분,일자,요일,시장,비고
0,휴장,2017/01/02,월,CME,New Year' Day
1,휴장,2017/04/14,금,CME,Good Friday
2,단축종료(03:15),2017/11/24,금,CME,추수감사절 다음날
3,휴장,2017/12/25,월,CME,Christmas Day


In [40]:
pd.read_excel('https://goo.gl/s2PgBx')

,구분,일자,요일,시장,비고
0,휴장,2017/04/14,금,EUREX,Good Friday
1,휴장,2017/04/17,월,EUREX,Easter Monday
2,휴장,2017/05/01,월,EUREX,Labour Day
3,휴장,2017/12/25,월,EUREX,Christmas Day
4,휴장,2017/12/26,화,EUREX,Boxing Day


# pandas-market-calendars

해외 시장의 개장일(market day)을 관리하기 위해 위 엑셀 데이터를 사용하기 보다는 pandas-market-calendars 라이브러를 권한다.

http://pandas-market-calendars.readthedocs.io

NYSE, CME, EUREX 등 시장에 대한 개장일과 개장 시간등을 관리할 수 있다.

#### 2017 https://fb.com/financedata

In [42]:
mdays = pd.date_range('2017-01-01', '2017-12-31', freq='B')

df_sample = pd.DataFrame(index=mdays)
df_sample.head(10)



""
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13


In [4]:
# 2017년 휴장일
data1 = pd.read_excel('https://goo.gl/cvnhy2')

hdays = data1['일자']

hdays = pd.to_datetime(hdays)

hdays.name = '일자'

df_mdays = pd.DataFrame(index=hdays)
df_mdays.head()


""
일자
2017-01-02
2017-04-14
2017-11-24
2017-12-25


In [5]:
import random
random.randint(1, 10)

5

In [7]:
import numpy as np
a = np.random.randn(100).cumsum()
a

array([  0.37936326,  -0.50585691,  -0.54229559,   0.78754604,
         1.07835755,   0.80547303,   1.30882329,  -1.60475444,
        -1.2711072 ,  -2.09717324,  -3.73887823,  -4.46145023,
        -3.22199227,  -3.5533198 ,  -2.61551787,  -3.99349774,
        -4.14743794,  -3.24590735,  -3.59322488,  -3.25719294,
        -3.16218636,  -3.69730342,  -3.57109514,  -1.72677277,
        -1.31740739,  -2.01839691,  -2.16716755,  -2.94746191,
        -3.24994746,  -3.96684987,  -3.25394017,  -3.00065486,
        -1.7817621 ,  -3.00174937,  -3.5690225 ,  -3.76690231,
        -4.09234865,  -4.79500264,  -6.23129573,  -5.35580506,
        -4.82971767,  -3.62094382,  -3.71251402,  -5.72831941,
        -6.0525061 ,  -5.15757564,  -6.183176  ,  -5.3452256 ,
        -4.95531973,  -4.68773547,  -3.07941811,  -3.00943328,
        -3.33473088,  -4.70010021,  -4.95267835,  -4.91543449,
        -5.06607534,  -6.01284418,  -6.01793429,  -7.87360445,
        -8.46359582,  -7.80565089,  -7.91784684,  -8.69

In [8]:
count = 0
for i in range(10000):
    for j in str(i):
        if j == '8':
            count += 1
            
print(count)

4000
